In [120]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [121]:
movies=pd.read_csv('Boltzmann_Machines/ml-1m/movies.dat',sep='::',
                  header=None,engine='python',
                  encoding='latin-1')

In [122]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [123]:
users=pd.read_csv('Boltzmann_Machines/ml-1m/users.dat',sep='::',
                  header=None,engine='python',
                  encoding='latin-1')

In [124]:
# 1st col user id 2nd col gender 3rd col age 4th 
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [125]:
ratings=pd.read_csv('Boltzmann_Machines/ml-1m/ratings.dat',sep='::',
                  header=None,engine='python',
                  encoding='latin-1')

In [126]:
# 1st col user id 2nd col movie id 3rd col rating 
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [127]:
movies.loc[357]

0                              361
1    It Could Happen to You (1994)
2                    Drama|Romance
Name: 357, dtype: object

In [128]:
training_set=pd.read_csv('Boltzmann_Machines/ml-100k/u1.base',delimiter='\t')

In [129]:
# 1ST COlumn is user no, 2nd column corrosponds to movies and 3rd col rating,
# 4th column is timestamp
training_set

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [130]:
training_set=np.array(training_set,dtype=int)

In [131]:
training_set.shape

(79999, 4)

In [132]:
test_set=pd.read_csv('Boltzmann_Machines/ml-100k/u1.test',delimiter='\t')

In [133]:
test_set=np.array(test_set,dtype='int')

In [134]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [135]:
training_set[79998]
# It means total number of users in training set is 943

array([      943,      1330,         3, 888692465])

In [136]:
# It means total number of users in test set is 462
test_set[19998]

array([      462,       682,         5, 886365231])

In [137]:
nb_users=int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies=int(max(max(training_set[:,1]),max(test_set[:,1])))

In [138]:
nb_movies,nb_users

(1682, 943)

In [139]:
def convert(data):
    new_data=[]
    for id_users in range(1,nb_users+1):
        id_movies=data[:,1][data[:,0]==id_users]
        id_ratings=data[:,2][data[:,0]==id_users]
        ratings=np.zeros(nb_movies)
        ratings[id_movies-1]=id_ratings
        new_data.append(list(ratings))
    return new_data


In [140]:
training_set=convert(training_set)
test_set=convert(test_set)

In [141]:
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

In [146]:
# converting the ratings into binary ratings 1(Liked) or 0 (Not Liked)
training_set[training_set==0]=-1
training_set[training_set==1]=0
training_set[training_set==2]=0
training_set[training_set>=3]=1

In [148]:
test_set[test_set==0]=-1
test_set[test_set==1]=0
test_set[test_set==2]=0
test_set[test_set>=3]=1

In [161]:
# nv=number of visible nodes, nh=number of visible nodes
class RBM():
    def __init__(self,nv,nh):
        self.W=torch.randn(nh,nv)
        
#       bias of hidden node
        self.a=torch.randn(1,nh)

#       bias of visible node
        self.b=torch.randn(1,nv)

#     samples of hidden nodes
    def sample_h(self,x):
#     product of two tensors by torch.mm()
        wx=torch.mm(x,self.W)
        activation=wx+self.a.expand_as(wx)
        p_h_given_v=torch.sigmoid(activation)
        return p_h_given_v,torch.bernoulli(p_h_given_v)
    
    
        
        
        
        
        
        